# Using Customized and External Models

Through the previous tutorials we've be using networks that are built off of cell-models that are downloaded from the [Allen Cell-Types Database](https://celltypes.brain-map.org/data). While the Allen Cell-Types Databases has a wide and continuing growing selection of optimized cell-type models to download and use, for many scientists it may not meet their needs. BMTK supports the ability to incorporate many different types and cell and synaptic models into their simulations. Including pre-built models from non-Allen Institute organizations as-well-as allowing users multiple ways to create their own customized models. 

In the following we will show users multiple ways they can import or create their own models.

## BioNet Cell models

This section will focus on single-cell spiking models, including condutance-based models, with multiple channels and/or compartments per model. We do not recommend using Rates-based models with BioNet and will likely not work.

### Hoc Models



#### Tutorial: Loading in a Simple Hoc Model


##### Step 1: 

**Note**: *This step has already been done in the current notebook and does not need to be repeated, but may be useful when intializing other examples*

First Thing we'll do is to download and unpack the model files from ModelDB, which for the Hay L5PC models can be found at  [https://modeldb.science/139653](https://modeldb.science/139653). At the site you can use the download button to download a zip file with the appropiate .hoc (cell model), .mod (ion channels), and .asc (morphologies) files.


<br>
<div>
  <img src="../images/modeldb_hay_screen.png" width="800" align="center" style="margin-left:5px"/>    
</div>
<br clear="left">

To run properly we will need to move the following files from the download zip file into our **components/** folder so BMTK will be able to find them during the simulation:
* Move **mod/** folder => **componets/mechanisms/mod/**.
* Move **morphologies/\*.asc** files => **components/morphologies/** folder.
* Move **models/L5PCtemplate.hoc** and **models/L5PCbiophys4.hoc** => **componets/templates/** folder (*note*: The zip file contains four different version of L5PCbiophys model stored in different hoc files. However only one can be loaded into NEURON at a time, hence we are only moving one of the files and not all of them).
 


The model contains customized ion channels in the **components/mechanisms/mod/** folder that require compiling using the `nrnivmod` command. In a shell or using jupyter notebook you'll need to run the following:

In [12]:
! cd components/mechanisms && nrnivmodl modfiles

/local1/.local/miniconda-23.10.0/bin:/local1/.local/miniconda-23.10.0/condabin:/local1/.local/miniconda-23.10.0/bin:/local1/apps/pycharm/bin:/usr/local/bin:/usr/local/sbin:/usr/lib64/qt-3.3/bin:/home/kaeld/perl5/bin:/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/bin:/sbin:/home/kaeld/.rvm/bin:/home/kaeld/.rvm/bin:/usr/pgsql-9.4/bin:/home/kaeld/bin


##### Step 2: Building the Network

Build a network using the downloaded L5PC Hay model is very similiar to how we built biophysically detailed network models in [previous chapters](../Ch3_multicells/3.%20Multipopulation%20biophysical.ipynb). We use the `add_nodes()` method to add a population of 1 or more L5PC `biophsyical` cells and use the `add_edges()` to connect them up with other cells. But due to the model type and parameters we will need to make some changes vs how we utilize Allen-Cell Type models

* We set **model_template**=`hoc:L5PCtemplate` instead of the Allen `ctdb:Biophys1` value. The name of the Hoc Template for loading this particular model comes from the components/templates/L5PCtemplate.hoc file
* In our Allen Cell-Types models we had set value **model_processing** to 'aibs_perisomatic', which told bmtk to call special post-intialization directive `aibs_perisomatic` which would modify each cell object in a specific manner. Although it is possible to create custom directives to update and modify each cell after the template has been loaded (see below), we don't want to do so and instead leave the model_processing attribute blank.
* In our Allen Cell-Types models we also had a **dynamics_params** option to would go through update and add cell parameters after the cell has been initialzed. However for our L5PC Hay model we have no current need to update the parameters (we'll instead use the parameter values that are stored in the .hoc files) we will not have any dynamics_params attributes.


    

In [15]:
from bmtk.builder.networks import NetworkBuilder

net = NetworkBuilder('cell')
net.add_nodes(
    N=10,
    model_type='biophysical',
    model_template='hoc:L5PCtemplate',
    morphology='cell3.asc'
)
net.build()
net.save(output_dir='network')

In [16]:
virt_exc = NetworkBuilder('virt_exc')
virt_exc.add_nodes(
    N=20,
    model_type='virtual',
    ei_type='exc'
)
conns = virt_exc.add_edges(
    source=virt_exc.nodes(),
    target=net.nodes(),
    connection_rule=12,
    model_template='Exp2Syn',
    dynamics_params='AMPA_ExcToExc.json',
    distance_range=[0.0, 1.0e20],
    target_sections=['soma', 'basal', 'apical'],
    delay=2.0,
    syn_weight=0.01
)

virt_exc.build()
virt_exc.save(output_dir='network')

##### Step 3: Running the simulation

Running the simulation with the download L5PC models is no different than running the simulations with the Allen Cell-Types Database cell models. BMTK will be able to use the configuration **components** sections along with the **model_template** and **morphology** values to automatically load in our 10 L5PC cells. The results for these cells, including spikes, Vm traces, etc; will be saved in the same format and paths as with our other examples.

In [17]:
from bmtk.simulator import bionet

bionet.reset()
# conf = bionet.Config.from_json('config.iclamp.json')
conf = bionet.Config.from_json('config.with_syns.json')
conf.build_env()

net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

numprocs=1
2024-04-29 16:58:19,169 [INFO] Created log file


INFO:NEURONIOUtils:Created log file


2024-04-29 16:58:19,247 [INFO] Building cells.


INFO:NEURONIOUtils:Building cells.


2024-04-29 16:58:30,738 [INFO] Building recurrent connections


...

INFO:NEURONIOUtils:    step:25000 t_sim:2500.00 ms


2024-04-29 17:01:59,371 [INFO]     step:30000 t_sim:3000.00 ms


INFO:NEURONIOUtils:    step:30000 t_sim:3000.00 ms


2024-04-29 17:01:59,403 [INFO] Simulation completed in 3.0 minutes, 28.49 seconds 


INFO:NEURONIOUtils:Simulation completed in 3.0 minutes, 28.49 seconds 


#### Tutorial: Overloading HOC Template initialization




###  NeuroML Models

### Building your own models from scratch